In [1]:
from atmos_functions import *
import xarray as xr
import odvc
from collections import OrderedDict

url_g1="http://tds.marine.rutgers.edu/thredds/dodsC/roms/espresso/2013_da/avg/ESPRESSO_Real-Time_v2_Averages_Best"
ds_g1 = xr.open_dataset(url_g1)
ds_g1 = ds_g1.chunk({})

url_g2 = "http://tds.marine.rutgers.edu/thredds/dodsC/roms/doppio/2017_da/avg/Averages_Best"
ds_g2 = xr.open_dataset(url_g2)
# Turn on chunking to activate dask and parallelize read/write
ds_g2 = ds_g2.chunk({})

In [2]:
def filter_by(self, **kwargs):
    selection = []
    for var_name, variable in self.variables.items():
        has_value_flag = False
        for attr_name, pattern in kwargs.items():
            attr_value = variable.attrs.get(attr_name)
            if ((callable(pattern) and pattern(attr_value))
                    or attr_value == pattern):
                has_value_flag = True
            else:
                has_value_flag = False
                break
        if has_value_flag is True:
            selection.append(var_name)
    return self[selection]

In [3]:
def get_formula_terms(var):
    formula_terms = OrderedDict()
    var = var.attrs.get("formula_terms")
    terms = [x.strip(":") for x in var.split()]
    for k, v in zip(terms[::2], terms[1::2]):
        formula_terms.update({k: v})
    return formula_terms

In [4]:
S = filter_by(ds_g2, standard_name="ocean_s_coordinate_g2")  # need to fix filter_by to find the coords
S = S["s_rho"]  # need to find this automagically

map_s = get_formula_terms(S)

In [5]:
my_map_s = {k.lower(): ds_g2[map_s[k]] for k, v in map_s.items()}

In [6]:
z = odvc.ocean_s_coordinate_g2(**my_map_s)
z

<xarray.DataArray (time: 669, eta_rho: 106, xi_rho: 242, s_rho: 40)>
dask.array<shape=(669, 106, 242, 40), dtype=float64, chunksize=(669, 106, 242, 40)>
Coordinates:
    lon_rho   (eta_rho, xi_rho) float64 dask.array<shape=(106, 242), chunksize=(106, 242)>
    lat_rho   (eta_rho, xi_rho) float64 dask.array<shape=(106, 242), chunksize=(106, 242)>
  * time      (time) datetime64[ns] 2017-11-02T12:00:00 ... 2019-09-01T12:00:00
    time_run  (time) datetime64[ns] dask.array<shape=(669,), chunksize=(669,)>
  * s_rho     (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
Dimensions without coordinates: eta_rho, xi_rho